# 20200611 Lab06: SOFTMAX Classification

In [1]:
import tensorflow as tf
import numpy as np

## Create Dataset

In [89]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

### 문제 1: 'Softmax Classification' 단계별로 이해하기

- 각 단계별로 존재하는 '???'을 채워주세요!
- '???'의 개수에 상관없이, 분류 모델이 제대로 작동할 수 있도록 셀(cell)을 작성해도 좋습니다.
- 단계별로 존재하는 주석을 참조하시면 문제 풀이에 도움이 될 수 있습니다.
- 텐서플로우의 모든 메소드를 암기하는 것이 꼭 필요한지도 모르겠고, 아직 불가능하다고 생각합니다. 다만 식이 전개되는 과정과, 어떤 변수가 어떤 스텝에서 어디에 할당되는지에 초점을 맞추고 문제를 풀어주시면 좋을 것 같습니다.

In [90]:
# convert into numpy and float format
x_data = np.array(x_data, dtype='float32')
y_data = np.array(y_data, dtype='float32')

nb_classes = 3 # assign number of classes

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


## Weight and Bias

In [131]:
W = tf.Variable(tf.random.normal([4, nb_classes]), name = 'weight') # Weight의 shape: (변수 수, 클래수 수)
b = tf.Variable(tf.random.normal([nb_classes]), name = 'bias') # bias의 shape: (클래스 수)
variables = [W, b]

print(W, b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.1418272 ,  0.09217681,  0.45604834],
       [-0.6689453 , -0.23584062,  1.364712  ],
       [ 0.76964015, -1.1848927 ,  0.3692249 ],
       [ 1.8074064 , -0.6818398 , -0.2481708 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.63958806, -0.19781014, -0.2723026 ], dtype=float32)>


## Softmax Regression

In [103]:
def hypothesis(X):
    return tf.matmul(X, W) + b

print(hypothesis(x_data))

tf.Tensor(
[[ -1.5600209  -1.2019196  -3.490603 ]
 [ -6.374506   -0.1434229  -7.9475317]
 [ -7.970931   -3.7607284 -11.147406 ]
 [-12.410716   -2.6771297 -15.665702 ]
 [ -7.8023987  -2.1412134 -14.204047 ]
 [-10.080752   -4.011161  -15.192174 ]
 [-10.206067   -2.432518  -16.744333 ]
 [-11.860335   -2.6736155 -19.407352 ]], shape=(8, 3), dtype=float32)


## Cost Function(Cross Entropy) & Gradient Descent
- 비용함수를 정의함에 있어 tensorflow의 keras.losses.categorical_crossentropy 메소드를 사용해도 좋습니다!

In [132]:
def logit_fn(X) :
    return tf.matmul(X, W) + b

def hypothesis(X) :
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y) :
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits,
                                                      from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def cost_fn2(X, Y) :
    logits = hypothesis(X)
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1)) 
    return cost

In [133]:
print(cost_fn(x_data, y_data))
print(cost_fn2(x_data, y_data))

tf.Tensor(10.664778, shape=(), dtype=float32)
tf.Tensor(10.664778, shape=(), dtype=float32)


In [116]:
def grad_fn(X,Y): # 비용함수를 최적화하기 위해 Gradient Descent 방식을 사용합니다
    with tf.GradientTape() as tape :
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

## Train

In [117]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_fn(x_data, y_data)
        optimizer.apply_gradients(zip(grads, variables))
        
        if (i==0) | ((i+1)%verbose == 0): # 손실이 줄어드는 과정을 출력
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X,Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 1.156445
Loss at epoch 100: 1.124177
Loss at epoch 200: 0.960401
Loss at epoch 300: 0.929183
Loss at epoch 400: 0.907474
Loss at epoch 500: 0.891111
Loss at epoch 600: 0.879189
Loss at epoch 700: 0.870478
Loss at epoch 800: 0.863900
Loss at epoch 900: 0.858724
Loss at epoch 1000: 0.854497
Loss at epoch 1100: 0.850939
Loss at epoch 1200: 0.847877
Loss at epoch 1300: 0.845196
Loss at epoch 1400: 0.842819
Loss at epoch 1500: 0.840689
Loss at epoch 1600: 0.838768
Loss at epoch 1700: 0.837023
Loss at epoch 1800: 0.835429
Loss at epoch 1900: 0.833969
Loss at epoch 2000: 0.832624


## Prediction Check

- Softmax function을 통과한 샘플 데이터의 확률값과 y_data에 기반해 예측된 분류값을 출력해주세요

In [119]:
# Prediction

sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32) # array 형태로 바꿔주세요

print(hypothesis(sample_data)) # sample_data의 각 범주에 대한 확률값을 출력해주세요
print(tf.argmax(hypothesis(sample_data), 1)) # 2차원 배열의 1번째 행에서, 예측된 범주를 출력해주세요(아래 출력 결과를 참고하시면 됩니다)

tf.Tensor([[-9.8245325 -3.3853965 -1.2555335]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [120]:
hypothesis([x_data[0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-4.2076755, -4.591408 ,  2.5465393]], dtype=float32)>

In [121]:
print(hypothesis(x_data)) # x_data의 각 범주에 대한 확률 값을 출력해주세요(필요하다면 변수를 할당해서 진행하세요)
print(tf.argmax(hypothesis([x_data[0]]), 1)) # x_data의 1번째 행에 대해, 예측된 범주를 출력해주세요(아래 출력 결과를 참고하시면 됩니다)
print(y_data[0]) # y_data의 실제 값을 출력해서, 바로 윗줄에서 예측한 범주와 일치하는지 확인하세요

tf.Tensor(
[[ -4.2076755   -4.591408     2.546539  ]
 [ -9.8245325   -3.3853967   -1.2555337 ]
 [-12.500203    -6.191369    -4.187501  ]
 [-18.377962    -5.2424574   -7.133137  ]
 [-16.622204    -1.5251722   -6.0002713 ]
 [-16.102001    -0.7660184  -12.416063  ]
 [-19.138807    -0.18014097 -10.063955  ]
 [-22.177214     0.65069246 -12.414768  ]], shape=(8, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
[0. 0. 1.]


## 문제2: 'softmax_classifier' 클라스 정의하기

- 위에서 단계적으로 구현한 함수를 바탕으로, 'softmax_classifier'의 메소드를 정의하고 주어진 샘플 데이터의 범주를 예측하세요

In [123]:
class softmax_classifier(tf.keras.Model):
    def __init__(self, n_features, nb_classes):
        self.W = tf.Variable(tf.random.normal([n_features, nb_classes]), name='weight')
        self.b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
        self.variable = [self.W, self.b]
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(X, Y)
            grads = tape.gradient(cost, self.variable)
            return grads
        
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variable))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))

### x_data, y_data를 참고해 nb_classes를 할당하고, 학습을 진행하세요

### 위에서 학습한 모델을 바탕으로, 주어진 sample data의 범주를 예측하세요
- 예측 과정을 class 내부에 정의하셔도 좋습니다

In [124]:
model = softmax_classifier(n_features=4, nb_classes=3)
model.fit(x_data, y_data)

Loss at epoch 1: 2.490724
Loss at epoch 500: 0.371896
Loss at epoch 1000: 0.224398
Loss at epoch 1500: 0.179537
Loss at epoch 2000: 0.149312


In [125]:
sample_data_2 = [[3,6,1,1]]
sample_data_2 = np.asarray(sample_data_2, dtype=np.float32)
tf.argmax(model.softmax_regression(sample_data_2), axis=1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([2], dtype=int64)>